In [43]:
import pandas as pd
import datetime

def missing_values(csv):
    df = pd.read_csv(csv)
    # grouped_df = df.groupby(["city"])
    return df[df.isna().any(axis=1)]



How many missing values we have and where


In [44]:
response = missing_values("../data/processed/weather_energy_data.csv")
print(response)
print(f"Missing Value Count: {len(response)}")

           date     city       state   TMAX  TMIN  \
771  2025-08-03  Phoenix     Arizona  112.0  84.0   
772  2025-08-03  Seattle  Washington  150.0   NaN   

                    respondent-name  timezone    value-units  Demand  \
771  Arizona Public Service Company  Mountain  megawatthours     NaN   
772              Seattle City Light   Pacific  megawatthours     NaN   

     Net generation  
771         78982.0  
772          7357.0  
Missing Value Count: 2


Any outliers (temperatures above 130°F or below -50°F, negative energy consumption)


In [45]:
df = pd.read_csv("../data/processed/weather_energy_data.csv")
df.head(1)

,date,city,state,TMAX,TMIN,respondent-name,timezone,value-units,Demand,Net generation
0,2025-03-01,Chicago,Illinois,34.0,22.0,"PJM Interconnection, LLC",Eastern,megawatthours,2110785.0,2206107.0


In [46]:
#filter weather report and let it return specific columns
filtered_df = df[(df["TMAX"]>130.0) | (df["TMIN"]<-50.0)]
bad_weather = filtered_df[["date", "city", "state", "TMAX", "TMIN"]]

# display result
# print(bad_weather)

In [47]:
#filter energy and return specific column
bad_energy = df[df["Demand"]<0][["date", "city", "state", "Demand", "value-units", "Net generation"]]

# display result
# print(bad_energy)

In [48]:
if not bad_weather.empty  or not bad_energy.empty:
    print(pd.concat([bad_weather, bad_energy], axis=0, ignore_index=True))
else:
    print(f"No outliers detected")


         date     city       state   TMAX  TMIN  Demand value-units  \
0  2025-08-03  Houston       Texas   98.0 -78.0     NaN         NaN   
1  2025-08-03  Seattle  Washington  150.0   NaN     NaN         NaN   

   Net generation  
0             NaN  
1             NaN  


reporting city freshness

In [49]:
import pandas as pd
import datetime

def report_city_freshness(csv_path, date_col="date", city_col="city"):
    # 1) Load and normalize
    df = pd.read_csv(csv_path) #header=None,
                    #  names=[date_col, "datatype", "station", "attrs", "value", city_col, "state"])
    # e.g. if your CSV has no header, adjust `names=` above

    # 2) Parse date and keep just the date part
    df[date_col] = pd.to_datetime(df[date_col]).dt.date

    today = datetime.date.today()

    reports = []
    # 3) For each city
    for city, grp in df.groupby(city_col):

        last_date = grp[date_col].max()
        if last_date < today:
            # 4) Missing range: day after last_date up to yesterday
            start_missing = last_date + datetime.timedelta(days=1)
            end_missing   = today #- datetime.timedelta(days=1)
            reports.append(
                f"{city} is missing data from {start_missing.isoformat()} to {end_missing.isoformat()}"
            )
        else:
            reports.append(f"{city} is up-to-date (has {last_date.isoformat()})")
    return reports



In [50]:
input_path = "../data/processed/weather_energy_data.csv"
response = report_city_freshness(input_path)

In [51]:
print(response)

['Chicago is missing data from 2025-08-03 to 2025-08-05', 'Houston is missing data from 2025-08-04 to 2025-08-05', 'New York is missing data from 2025-08-03 to 2025-08-05', 'Phoenix is missing data from 2025-08-04 to 2025-08-05', 'Seattle is missing data from 2025-08-04 to 2025-08-05']
